In [ ]:
## Load in diffusion pusht trained out our data (this has been mounted on the docker)

from lerobot.common.policies.act.modeling_act import ACTPolicy

inference_time_s = 120
fps = 10
device = "cuda"  # TODO: On Mac, use "mps" or "cpu"

ckpt_path =  "/opt/pretrained_model"
policy = ACTPolicy.from_pretrained(ckpt_path)

In [ ]:
#move to cuda
policy.to(device)

In [ ]:
#setup the robot config
from lerobot.common.robot_devices.motors.configs import DynamixelMotorsBusConfig
from lerobot.common.robot_devices.motors.dynamixel import DynamixelMotorsBus

leader_config = DynamixelMotorsBusConfig(
    port="/dev/ttyACM_kochleader",
    motors={
        # name: (index, model)
        
        "shoulder_pan": (1, "xl330-m077"),
        "shoulder_lift": (2, "xl330-m077"),
        "elbow_flex": (3, "xl330-m077"),
        "wrist_flex": (4, "xl330-m077"),
        "wrist_roll": (5, "xl330-m077"),
        "gripper": (6, "xl330-m077"),
    },
)

follower_config = DynamixelMotorsBusConfig(
    port="/dev/ttyACM_kochfollower",
    motors={
        # name: (index, model)
        "shoulder_pan": (1, "xl430-w250"),
        "shoulder_lift": (2, "xl430-w250"),
        "elbow_flex": (3, "xl330-m288"),
        "wrist_flex": (4, "xl330-m288"),
        "wrist_roll": (5, "xl330-m288"),
        "gripper": (6, "xl330-m288"),
    },
)

from lerobot.common.robot_devices.cameras.configs import OpenCVCameraConfig
from lerobot.common.robot_devices.cameras.opencv import OpenCVCamera
from lerobot.common.robot_devices.robots.configs import KochRobotConfig
from lerobot.common.robot_devices.robots.manipulator import ManipulatorRobot

config_cam_1 = OpenCVCameraConfig(
                camera_index = "/dev/video0",
                fps=10,
                width= 1280,
                height = 720,
                color_mode = 'rgb'
            )
config_cam_2 = OpenCVCameraConfig(
                camera_index = "/dev/video2",
                fps=10,
                width= 1280,
                height = 720,
                color_mode = 'rgb'
            )


robot = ManipulatorRobot(
    KochRobotConfig(
        leader_arms={"main": leader_config},
        follower_arms={"main": follower_config},
        calibration_dir=".cache/calibration/koch",
        cameras={
            "logitech1": config_cam_1,
            "logitech2": config_cam_2,
        },
    )
)
robot.connect()

In [ ]:
#run the policy
import time
import torch
import time
from lerobot.scripts.control_robot import busy_wait


for _ in range(inference_time_s * fps):
    start_time = time.perf_counter()

    # Read the follower state and access the frames from the cameras
    observation = robot.capture_observation()

    # Convert to pytorch format: channel first and float32 in [0,1]
    # with batch dimension
    for name in observation:
        if "image" in name:
            observation[name] = observation[name].type(torch.float32) / 255
            observation[name] = observation[name].permute(2, 0, 1).contiguous()
        observation[name] = observation[name].unsqueeze(0)
        observation[name] = observation[name].to(device)

    # Compute the next action with the policy
    # based on the current observation
    action = policy.select_action(observation)
    # Remove batch dimension
    action = action.squeeze(0)
    # Move to cpu, if not already the case
    action = action.to("cpu")
    # Order the robot to move
    robot.send_action(action)

    dt_s = time.perf_counter() - start_time
    busy_wait(1 / fps - dt_s)